In [232]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report
from fairlearn.reductions import GridSearch, DemographicParity
from fairlearn.metrics import metric_frame, selection_rate


ImportError: cannot import name 'metric_frame' from 'fairlearn.metrics' (/Users/aymanatmani/miniforge3/lib/python3.10/site-packages/fairlearn/metrics/__init__.py)

In [ ]:
# Load the dataset
file_path = '../DataSet/data_participant.csv'
data = pd.read_csv(file_path)

In [ ]:
# Encode categorical variables
label_encoder = LabelEncoder()
categorical_cols = ['SEX', 'EDUCATION', 'MARRIAGE']
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])


In [ ]:

# Remove rows with missing target values
train_data = data.dropna(subset=['default-payment-next-month'])
test_data = data[data['default-payment-next-month'].isnull()]

In [ ]:

# Split data into X (features) and y (target)
X = train_data.drop(columns=['ID', 'default-payment-next-month'])
y = train_data['default-payment-next-month']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

In [233]:
# model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
# model.fit(X_train, y_train)

# # Predict and evaluate performance
# y_pred = model.predict(X_test)

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))


# Identify Sensitive Feature
sensitive_feature = X_train['SEX']  # Assuming SEX is a sensitive feature

# Set up GridSearch for DemographicParity constraint
sweep = GridSearch(GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
                   constraints=DemographicParity(),
                   grid_size=70)


In [234]:

# Train with constraint
sweep.fit(X_train, y_train, sensitive_features=sensitive_feature)


AttributeError: 'GridSearch' object has no attribute '_select_best_model_index'

In [244]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score

best_model = None
best_balance = float('inf')

for predictor in sweep.predictors_:
    y_pred = predictor.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Step 1: Compute disparity
    metric_frame = MetricFrame(accuracy_score, y_test, y_pred, sensitive_features=X_test['SEX'])
    # Assuming 0 and 1 are the encoded values for male and female in SEX
    disparity = abs(metric_frame.by_group[0] - metric_frame.by_group[1])

    # Step 2: Combine accuracy and disparity
    alpha = 0.5  # Adjust this value based on your priorities
    balance = alpha * (1 - accuracy) + (1 - alpha) * disparity
    
    if balance < best_balance:
        best_balance = balance
        best_model = predictor



/Users/aymanatmani/miniforge3/lib/python3.10/site-packages/fairlearn/metrics/_metric_frame.py:77: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(
/Users/aymanatmani/miniforge3/lib/python3.10/site-packages/fairlearn/metrics/_metric_frame.py:77: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(
/Users/aymanatmani/miniforge3/lib/python3.10/site-packages/fairlearn/metrics/_metric_frame.py:77: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(
/Users/aymanatmani/miniforge

In [245]:
from fairlearn.metrics import MetricFrame, selection_rate

metric_frame_instance = MetricFrame(selection_rate,
                                    y_true=y_test,
                                    y_pred=y_pred,
                                    sensitive_features=X_test['SEX'])

print("Selection Rate Group Summary:")
print(metric_frame_instance.by_group)



X_val = test_data.drop(columns=['ID', 'default-payment-next-month'])
# Predict on Validation Set
y_val = best_model.predict(X_val)
y_val = y_val.astype(int)  # Convert to integers
test_data['default-payment-next-month'] = y_val

Selection Rate Group Summary:
SEX
0    1.0
1    0.0
Name: selection_rate, dtype: float64


/Users/aymanatmani/miniforge3/lib/python3.10/site-packages/fairlearn/metrics/_metric_frame.py:77: FutureWarning: You have provided 'metrics' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(
/var/folders/5z/d82bj78x63vb86pgwwtjf51m0000gn/T/ipykernel_18797/1979757879.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['default-payment-next-month'] = y_val


In [246]:
combined_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

final_data = pd.read_csv(file_path)

final_data['default-payment-next-month'] = combined_data['default-payment-next-month'].astype(int)



In [247]:
final_data.to_csv('submission_gb.csv', index=False)